In [0]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image  , ImageDraw 
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import os
import json
#from google.colab import drive
#drive.mount('/content/drive')


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## **Test1 Raw Hand Face Pose As Depth**


In [0]:

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((224,224)),
        #transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
        #transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ]),
    'validation':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]),
    'test':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]),
}

class MyImageFolder(datasets.ImageFolder):
    
    def __init__(self, root, transform=None):
        super(MyImageFolder,self).__init__(root)

        self.transform = transform

    def __getitem__(self, index):

        #imgs = self.imgs[index]
        #print(imgs)

        path, target = self.samples[index]
        #sample = self.loader(path)
        #print("Sample: ",sample)
        

        hm_transform = transforms.Compose([
                          transforms.Resize((224,224)),
                          transforms.ToTensor()

                                ])
        self.transform = hm_transform
        #print(path)
        #print(type(path))
        #all_images = []
        #all_labels = []
        #all_hm_face = []
        #all_hm_hand = []
        # target_acquired - 0: image path -- 1: label
        #for i in len(path):

        img_path = path
        #print("--i: %d , TA: %s"%(i,target_acquired))

        
        #img_path = target_acquired
        #label =  target_acquired[1]

        #print(img_path)

        image = Image.open(img_path)
        img_w , img_h = image.size

    
        filename = os.path.basename(img_path)
        ann_path = img_path.replace(filename, filename.split('.')[0])
        hm_face , hm_hand = self.get_heatmap(ann_path + '.txt',img_w,img_h)  
        hm_pose = self.get_heatmap_pose(ann_path + '.json',img_w,img_h)
        #image = np.dstack((image, hm_face))
        cpath  = img_path.replace(filename, filename.split('.')[0] + '-bbox.txt')
         
        image,hm_face,hm_hand,hm_pose = self.crop_text(cpath,image,hm_face,hm_hand,hm_pose)
        # Transforms
        if self.transform is not None:
            
            image = self.transform(image)
            hm_face = hm_transform(hm_face)
            hm_hand = hm_transform(hm_hand)
            hm_pose = torch.stack([ hm_transform(x) for x in hm_pose ], dim=1).squeeze(0)


        #print( hm_pose.size() )
        image = torch.cat((image,hm_face,hm_hand,hm_pose))
       
        #print(image.size())
        #if self.target_transform is not None:
        #    target = self.target_transform(target)
        
        
        # For Face
            #image = np.dstack((image, hm_face))

            #all_images.append(img_path)
            #all_labels.append(label)
            #all_hm_face.append(hm_face)
            #all_hm_hand.append(hm_hand)

        #return path, all_hm_face, all_hm_hand, target
        
        return image, target

    def crop_text(self,path,image,hm_face,hm_hand,hm_pose):
        try:
              fr = open(path)
              # print ("fr",cat+"/yolo/" + imagePath.split("/")[-1].split('.')[0]+".txt")

              line = fr.readline().split(" ")

              width, height = image.size 
        
              # Setting the points ffloor cropped image
              b_w = float(line[3]) * width
              b_h = float(line[4]) * height

              t_left = (float(line[1]) * width) - (b_w/2)
              t_right = (float(line[2]) * height) - (b_h/2)

              b_left = (float(line[1]) * width) + (b_w/2)
              b_right = (float(line[2]) * height) + (b_h/2)

              # print("w",width," h",height)

              # print (t_left)
              # print (t_right)
              # print (b_left)
              # print (b_right)

              image = image.crop((t_left, t_right, b_left, b_right))
              
              hm_face = hm_face.crop((t_left, t_right, b_left, b_right))
              hm_hand = hm_hand.crop((t_left, t_right, b_left, b_right))
              
              for i in range(len(hm_pose)):
                  hm_pose[i] = hm_pose[i].crop((t_left, t_right, b_left, b_right))

        except FileNotFoundError:
              print("FileNotFoundError: " + path)
              #pass
        return image ,hm_face,hm_hand,hm_pose

    def get_heatmap(self,ann_path, img_w , img_h):
        
        hm_0 = Image.new('L', (img_w, img_h))
        hm_1 = Image.new('L', (img_w, img_h))
        
        try:

            with open(ann_path) as f:

                for line in f.readlines():

                    if line == '\n':
                        continue

                    line = line.split(' ')

                    cx = float(line[1])
                    cy = float(line[2])
                    w = float(line[3])
                    h = float(line[4])

                    xmin =  (cx - (w/2)) * img_w
                    ymin = (cy - (h/2)) * img_h
                    xmax = (cx + (w/2)) * img_w
                    ymax = (cy + (h/2)) * img_h

                    if line[0] == '0':
                        hmDraw = ImageDraw.Draw(hm_0)   
                    else:
                        hmDraw = ImageDraw.Draw(hm_1)   
                    
                    hmDraw.rectangle([xmin,ymin,xmax,ymax], fill ="#ffffff")
        except FileNotFoundError:
            #print("FileNotFoundError: %s"%ann_path)
            pass

        return hm_0, hm_1

    def get_heatmap_pose(self,ann_path, img_w, img_h):
        
        heatmaps = [ Image.new('L', (img_w, img_h)) for x in range(17) ]

        try:
            with open(ann_path,'r') as f:
                data = json.load(f)
            #print(ann_path)
            #print(data)  
            try:
                keypoints = data[0]['keypoints']
            except IndexError:
                return heatmaps

            #print(keypoints)
            # TODO check for two key mappings -- working

            for i in range(0,len(keypoints),3):

                xmin = keypoints[i] - 5
                ymin = keypoints[i+1] - 5
                xmax = keypoints[i] + 5
                ymax = keypoints[i+1] + 5

                hmDraw = ImageDraw.Draw(heatmaps[i//3]) 
                hmDraw.rectangle([xmin,ymin,xmax,ymax], fill ="#ffffff")

        except FileNotFoundError:
            #print("FileNotFoundError: %s"%ann_path)
            pass

        return heatmaps






input_path = "/content/drive/My Drive/Resnet_Beta-Version/dataset/alpha/Fall/" # check path for FALL or CHOKE 
image_datasets = {
    'train': 
    MyImageFolder(input_path + 'train', data_transforms['train']),
    'validation': 
    MyImageFolder(input_path + 'val', data_transforms['validation']),
    'test': 
    MyImageFolder(input_path + 'test', data_transforms['test'])
   
    }

dataloaders = {
    'train':
    torch.utils.data.DataLoader(image_datasets['train'],
                                batch_size=64,
                                shuffle=True,
                                num_workers=2),  # for Kaggle
    'validation':
    torch.utils.data.DataLoader(image_datasets['validation'],
                                batch_size=1,
                                shuffle=True,
                                num_workers=2) , # for Kaggle
  'test':
    torch.utils.data.DataLoader(image_datasets['test'],
                                batch_size=1,
                                shuffle=True,
                                num_workers=2)  # for Kaggle

}


In [0]:
epoch = 80
device = 'cuda'
model = torch.load('/content/drive/My Drive/Resnet_Beta-Version/weights/gamma/Fall/raw-face-hand-pose/rfhp_epoch_%d.pth'%(epoch)).to(device) #correct path please  for FALL or CHOKE 
optimizer = optim.Adam(model.fc.parameters(),lr=0.01)
model.eval()
predictions = []
target = []

iteration = 0

for inputs , labels in dataloaders['validation']:

    #if iteration == 150:
    #  break
    iteration += 1
    inputs = inputs.to(device)
    labels = labels.to(device)

    
    with torch.set_grad_enabled(False):
        pred_logits_tensorFalling = model(inputs)
        predictions.append(pred_logits_tensorFalling)
        target.append(labels)

    print("iteration"+str(iteration))


iteration1
iteration2
iteration3
iteration4
iteration5
iteration6
iteration7
iteration8
iteration9
iteration10
iteration11
iteration12
iteration13
iteration14
iteration15
iteration16
iteration17
iteration18
iteration19
iteration20
iteration21
iteration22
iteration23
iteration24
iteration25
iteration26
iteration27
iteration28
iteration29
iteration30
iteration31
iteration32
iteration33
iteration34
iteration35
iteration36
iteration37
iteration38
iteration39
iteration40
iteration41
iteration42
iteration43
iteration44
iteration45
iteration46
iteration47
iteration48
iteration49
iteration50
iteration51
iteration52
iteration53
iteration54
iteration55
iteration56
iteration57
iteration58
iteration59
iteration60
iteration61
iteration62
iteration63
iteration64
iteration65
iteration66
iteration67
iteration68
iteration69
iteration70
iteration71
iteration72
iteration73
iteration74
iteration75
iteration76
iteration77
iteration78
iteration79
iteration80
iteration81
iteration82
iteration83
iteration84
i

In [0]:
out = []

for pred in predictions:

    out.append( torch.argmax(pred).item()) 

acc = 0
for i,tar in enumerate(target):
    if tar.item() == out[i]:
      acc += 1

print("acc", acc)
print("total",len(out))
print(acc/len(out))

acc 368
total 500
0.736


## TEST 2 (Raw + Hand)

In [0]:


normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((224,224)),
        #transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
        #transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        #normalize
    ]),
    'validation':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        #normalize
    ]),
    'test':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        #normalize
    ]),
}
class MyImageFolder(datasets.ImageFolder):
    
    def __init__(self, root, transform=None):
        super(MyImageFolder,self).__init__(root)

        self.transform = transform

    def __getitem__(self, index):

        #imgs = self.imgs[index]
        #print(imgs)

        path, target = self.samples[index]
        #sample = self.loader(path)
        #print("Sample: ",sample)
        

        hm_transform = transforms.Compose([
                          transforms.Resize((224,224)),
                          transforms.ToTensor()

                                ])

        #print(path)
        #print(type(path))
        #all_images = []
        #all_labels = []
        #all_hm_face = []
        #all_hm_hand = []
        # target_acquired - 0: image path -- 1: label
        #for i in len(path):

        img_path = path
        #print("--i: %d , TA: %s"%(i,target_acquired))

        
        #img_path = target_acquired
        #label =  target_acquired[1]

        #print(img_path)

        image = Image.open(img_path)
        img_w , img_h = image.size

    
        filename = os.path.basename(img_path)
        ann_path = img_path.replace(filename, filename.split('.')[0] + '.txt')
        hm_face , hm_hand = self.get_heatmap(ann_path,img_w,img_h)  

        #image = np.dstack((image, hm_face))
        cpath  = img_path.replace(filename, filename.split('.')[0] + '-bbox.txt')
            
        image,hm_hand =  self.crop_text(cpath,image,hm_hand)
        
        # Transforms
        if self.transform is not None:
            
            image = self.transform(image)
            hm_hand = hm_transform(hm_hand)
        
        image = torch.cat((image,hm_hand))
        #if self.target_transform is not None:
        #    target = self.target_transform(target)
        
        
        # For Face
            #image = np.dstack((image, hm_face))

            #all_images.append(img_path)
            #all_labels.append(label)
            #all_hm_face.append(hm_face)
            #all_hm_hand.append(hm_hand)

        #return path, all_hm_face, all_hm_hand, target
        
        return image, target

    def get_heatmap(self,ann_path, img_w , img_h):
        
        hm_0 = Image.new('L', (img_w, img_h))
        hm_1 = Image.new('L', (img_w, img_h))
        
        try:

            with open(ann_path) as f:

                for line in f.readlines():

                    if line == '\n':
                        continue

                    line = line.split(' ')

                    cx = float(line[1])
                    cy = float(line[2])
                    w = float(line[3])
                    h = float(line[4])

                    xmin =  (cx - (w/2)) * img_w
                    ymin = (cy - (h/2)) * img_h
                    xmax = (cx + (w/2)) * img_w
                    ymax = (cy + (h/2)) * img_h

                    if line[0] == '0':
                        hmDraw = ImageDraw.Draw(hm_0)   
                    else:
                        hmDraw = ImageDraw.Draw(hm_1)   
                    
                    hmDraw.rectangle([xmin,ymin,xmax,ymax], fill ="#ffffff")
        except FileNotFoundError:
            print("FileNotFoundError: %s"%ann_path)
            #pass

        return hm_0, hm_1

    def crop_text(self,path,image,heatmap):
        try:
              fr = open(path)
              # print ("fr",cat+"/yolo/" + imagePath.split("/")[-1].split('.')[0]+".txt")

              line = fr.readline().split(" ")

              width, height = image.size 
        
              # Setting the points ffloor cropped image
              b_w = float(line[3]) * width
              b_h = float(line[4]) * height

              t_left = (float(line[1]) * width) - (b_w/2)
              t_right = (float(line[2]) * height) - (b_h/2)

              b_left = (float(line[1]) * width) + (b_w/2)
              b_right = (float(line[2]) * height) + (b_h/2)

              # print("w",width," h",height)

              # print (t_left)
              # print (t_right)
              # print (b_left)
              # print (b_right)

              image = image.crop((t_left, t_right, b_left, b_right))
              heatmap = heatmap.crop((t_left, t_right, b_left, b_right))
        except FileNotFoundError:
              print("FileNotFoundError: " + path)
              #pass
        return image ,heatmap


input_path = "/content/drive/My Drive/Resnet_Beta-Version/dataset/alpha/Fall/"

image_datasets = {
    'train': 
    #datasets.ImageFolder(input_path + 'train', data_transforms['train']),
    #MyImageFolder(input_path + 'train', data_transforms['train']),
    #ImageFolderDataset(input_path + 'train',data_transforms['train'] ),
    #FallingDataset(input_path+'train',data_transforms['train']),
    'validation': 
    #MyImageFolder(input_path + 'val', data_transforms['validation']),
     'test': 
    MyImageFolder(input_path + 'test', data_transforms['test'])
    #ImageFolderDataset(input_path + 'val',data_transforms['validation'] )
}

dataloaders = {
    'train':
    torch.utils.data.DataLoader(image_datasets['train'],
                                batch_size=64,
                                shuffle=True,
                                num_workers=1),  # for Kaggle
    'validation':
    torch.utils.data.DataLoader(image_datasets['validation'],
                                batch_size=1,
                                shuffle=False,
                                num_workers=1),  # for Kaggle
     'test':
    torch.utils.data.DataLoader(image_datasets['test'],
                                batch_size=1,
                                shuffle=False,
                                num_workers=1)  # for Kaggle
}

In [0]:
image_datasets['train'].classes


['Choking', 'NonChoking']

In [0]:

epoch = 80
device = 'cuda'
model = torch.load('/content/drive/My Drive/Resnet_Beta-Version/weights/alpha/Fall/raw-hand/epoch_%d.pth'%(epoch)).to(device)
optimizer = optim.Adam(model.fc.parameters(),lr=0.01)
model.eval()
predictions = []
target = []

iteration = 0

for inputs , labels in dataloaders['validation']:

    #if iteration == 150:
    #  break
    iteration += 1
    inputs = inputs.to(device)
    labels = labels.to(device)
    #imgplot = plt.imshow(inputs)
  
    with torch.set_grad_enabled(False):
        pred_logits_tensorFalling = model(inputs)
        predictions.append(pred_logits_tensorFalling)
        #print ("pred_logits_tensorFalling "+ str(pred_logits_tensorFalling))
        #print ("LABELS :" +str(labels))
        target.append(labels)

    #print("iteration"+str(iteration))


FileNotFoundError: /content/drive/My Drive/Resnet_Beta-Version/dataset/alpha/Choke/val/Choking/VID20200419003843_128.txt
FileNotFoundError: /content/drive/My Drive/Resnet_Beta-Version/dataset/alpha/Choke/val/Choking/VID20200419003843_188.txt
FileNotFoundError: /content/drive/My Drive/Resnet_Beta-Version/dataset/alpha/Choke/val/Choking/VID20200419003913_280.txt
FileNotFoundError: /content/drive/My Drive/Resnet_Beta-Version/dataset/alpha/Choke/val/Choking/VID20200419004347_220.txt
FileNotFoundError: /content/drive/My Drive/Resnet_Beta-Version/dataset/alpha/Choke/val/Choking/VID20200419010234_216.txt
FileNotFoundError: /content/drive/My Drive/Resnet_Beta-Version/dataset/alpha/Choke/val/Choking/VID20200419011252_72.txt
FileNotFoundError: /content/drive/My Drive/Resnet_Beta-Version/dataset/alpha/Choke/val/Choking/VID20200419011552_156.txt
FileNotFoundError: /content/drive/My Drive/Resnet_Beta-Version/dataset/alpha/Choke/val/Choking/VID20200419011552_212.txt
FileNotFoundError: /content/drive

In [0]:
out = []

for pred in predictions:

    out.append( torch.argmax(pred).item()) 
print (out)
acc = 0
for i,tar in enumerate(target): 
    print ("["+str (tar.item())+","+ str (out[i])+"]")
    if tar.item() == out[i]:
      
      acc += 1

print("acc", acc)
print("total",len(out))
print(acc/len(out))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 

## Test 3 .Raw Face Hand

In [0]:

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((224,224)),
        #transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
        #transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ]),
    'validation':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]),
    'test':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]),

}

class MyImageFolder(datasets.ImageFolder):
    
    def __init__(self, root, transform=None):
        super(MyImageFolder,self).__init__(root)

        self.transform = transform

    def __getitem__(self, index):

        #imgs = self.imgs[index]
        #print(imgs)

        path, target = self.samples[index]
        #sample = self.loader(path)
        #print("Sample: ",sample)
        

        hm_transform = transforms.Compose([
                          transforms.Resize((224,224)),
                          transforms.ToTensor()

                                ])

        #print(path)
        #print(type(path))
        #all_images = []
        #all_labels = []
        #all_hm_face = []
        #all_hm_hand = []
        # target_acquired - 0: image path -- 1: label
        #for i in len(path):

        img_path = path
        #print("--i: %d , TA: %s"%(i,target_acquired))

        
        #img_path = target_acquired
        #label =  target_acquired[1]

        #print(img_path)

        image = Image.open(img_path)
        img_w , img_h = image.size

    
        filename = os.path.basename(img_path)
        ann_path = img_path.replace(filename, filename.split('.')[0] + '.txt')
        hm_face , hm_hand = self.get_heatmap(ann_path,img_w,img_h)  

        #image = np.dstack((image, hm_face))

        
        # Transforms
        if self.transform is not None:
            
            image = self.transform(image)
            hm_face = hm_transform(hm_face)
            hm_hand = hm_transform(hm_hand)

        image = torch.cat((image,hm_face,hm_hand))
        #if self.target_transform is not None:
        #    target = self.target_transform(target)
        
        
        # For Face
            #image = np.dstack((image, hm_face))

            #all_images.append(img_path)
            #all_labels.append(label)
            #all_hm_face.append(hm_face)
            #all_hm_hand.append(hm_hand)

        #return path, all_hm_face, all_hm_hand, target
        
        return image, target

    def get_heatmap(self,ann_path, img_w , img_h):
        
        hm_0 = Image.new('L', (img_w, img_h))
        hm_1 = Image.new('L', (img_w, img_h))
        
        try:

            with open(ann_path) as f:

                for line in f.readlines():

                    if line == '\n':
                        continue

                    line = line.split(' ')

                    cx = float(line[1])
                    cy = float(line[2])
                    w = float(line[3])
                    h = float(line[4])

                    xmin =  (cx - (w/2)) * img_w
                    ymin = (cy - (h/2)) * img_h
                    xmax = (cx + (w/2)) * img_w
                    ymax = (cy + (h/2)) * img_h

                    if line[0] == '0':
                        hmDraw = ImageDraw.Draw(hm_0)   
                    else:
                        hmDraw = ImageDraw.Draw(hm_1)   
                    
                    hmDraw.rectangle([xmin,ymin,xmax,ymax], fill ="#ffffff")
        except FileNotFoundError:
            print("FileNotFoundError: %s"%ann_path)
            #pass

        return hm_0, hm_1
"""
Test Code


path, target = self.samples[index]
sample = self.loader(path)
if self.transform is not None:
  sample = self.transform(sample)
if self.target_transform is not None:
  target = self.target_transform(target)
# this is where you load your resnet data
resnet_path = os.path.join(os.path.splitext(path)[0], '.pth')  # replace image extension with .pth
resnet = torch.load(resnet_path)  # load the stored features
return sample, resnet, target
"""
#dataset = MyImageFolder("falling_dataset/val/")
#re, labels = dataset[1:3]
#print(re)
#print(labels)
#re[2].save('pred_face.jpeg')
#re[3].save('pred_hand.jpeg')

input_path = "/content/drive/My Drive/Resnet_Beta-Version/dataset/alpha/Choke/"

image_datasets = {
    'train': 
    #datasets.ImageFolder(input_path + 'train', data_transforms['train']),
    MyImageFolder(input_path + 'train', data_transforms['train']),
    #ImageFolderDataset(input_path + 'train',data_transforms['train'] ),
    #FallingDataset(input_path+'train',data_transforms['train']),
    'validation': 
    MyImageFolder(input_path + 'val', data_transforms['validation']),
     'test': 
    MyImageFolder(input_path + 'test', data_transforms['test'])
    #ImageFolderDataset(input_path + 'val',data_transforms['validation'] )
}

dataloaders = {
    'train':
    torch.utils.data.DataLoader(image_datasets['train'],
                                batch_size=64,
                                shuffle=True,
                                num_workers=1),  # for Kaggle
    'validation':
    torch.utils.data.DataLoader(image_datasets['validation'],
                                batch_size=1,
                                shuffle=False,
                                num_workers=1),  # for Kaggle
     'test':
    torch.utils.data.DataLoader(image_datasets['test'],
                                batch_size=1,
                                shuffle=False,
                                num_workers=1)  # for Kaggle
}

In [0]:

epoch = 85
device = 'cuda'
model = torch.load('/content/drive/My Drive/Resnet_Beta-Version/weights/alpha/Choke/raw-face-hand/epoch_%d.pth'%(epoch)).to(device)
optimizer = optim.Adam(model.fc.parameters(),lr=0.01)
model.eval()
predictions = []
target = []

iteration = 0

for inputs , labels in dataloaders['validation']:

    #if iteration == 150:
    #  break
    iteration += 1
    inputs = inputs.to(device)
    labels = labels.to(device)

    
    with torch.set_grad_enabled(False):
        pred_logits_tensorFalling = model(inputs)
        predictions.append(pred_logits_tensorFalling)
        target.append(labels)

    #print("iteration"+str(iteration))


FileNotFoundError: /content/drive/My Drive/Resnet_Beta-Version/dataset/alpha/Choke/val/Choking/VID20200419003843_128.txt
FileNotFoundError: /content/drive/My Drive/Resnet_Beta-Version/dataset/alpha/Choke/val/Choking/VID20200419003843_188.txt
FileNotFoundError: /content/drive/My Drive/Resnet_Beta-Version/dataset/alpha/Choke/val/Choking/VID20200419003913_280.txt
FileNotFoundError: /content/drive/My Drive/Resnet_Beta-Version/dataset/alpha/Choke/val/Choking/VID20200419004347_220.txt
FileNotFoundError: /content/drive/My Drive/Resnet_Beta-Version/dataset/alpha/Choke/val/Choking/VID20200419010234_216.txt
FileNotFoundError: /content/drive/My Drive/Resnet_Beta-Version/dataset/alpha/Choke/val/Choking/VID20200419011252_72.txt
FileNotFoundError: /content/drive/My Drive/Resnet_Beta-Version/dataset/alpha/Choke/val/Choking/VID20200419011552_156.txt
FileNotFoundError: /content/drive/My Drive/Resnet_Beta-Version/dataset/alpha/Choke/val/Choking/VID20200419011552_212.txt
FileNotFoundError: /content/drive

In [0]:
out = []

for pred in predictions:
    
    out.append( torch.argmax(pred).item())
    print (pred)

acc = 0
for i,tar in enumerate(target):
    if tar.item() == out[i]:
       acc += 1
   
    #print ("Target,predicted["+str (tar.item())+","+ str (out[i])+"]")
     
print("acc", acc)
print("total",len(out))
print(acc/len(out))
print (out)
print (target)

tensor([[1.0000e+00, 3.4920e-11]], device='cuda:0')
tensor([[1.0000e+00, 1.4018e-09]], device='cuda:0')
tensor([[1.0000e+00, 1.3344e-08]], device='cuda:0')
tensor([[1.0000e+00, 4.5016e-09]], device='cuda:0')
tensor([[1.0000e+00, 1.2420e-08]], device='cuda:0')
tensor([[1.0000e+00, 1.4097e-10]], device='cuda:0')
tensor([[1.0000e+00, 3.5291e-10]], device='cuda:0')
tensor([[1.0000e+00, 7.9847e-09]], device='cuda:0')
tensor([[1.0000e+00, 6.2599e-09]], device='cuda:0')
tensor([[1.0000e+00, 1.4662e-08]], device='cuda:0')
tensor([[1.0000e+00, 2.2811e-08]], device='cuda:0')
tensor([[1.0000e+00, 2.3289e-08]], device='cuda:0')
tensor([[1.0000e+00, 9.8461e-09]], device='cuda:0')
tensor([[1.0000e+00, 4.4211e-08]], device='cuda:0')
tensor([[1.0000e+00, 8.9277e-09]], device='cuda:0')
tensor([[1.0000e+00, 6.5032e-06]], device='cuda:0')
tensor([[1.0000e+00, 1.0294e-06]], device='cuda:0')
tensor([[1.0000e+00, 3.6642e-07]], device='cuda:0')
tensor([[1.0000e+00, 4.5310e-08]], device='cuda:0')
tensor([[1.0

323


#  Test 4 (Raw + Face )

In [0]:

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]),
    'validation':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]),
    'test':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]),
}

class MyImageFolder(datasets.ImageFolder):
    
    def __init__(self, root, transform=None):
        super(MyImageFolder,self).__init__(root)

        self.transform = transform

    def __getitem__(self, index):

        #imgs = self.imgs[index]
        #print(imgs)

        path, target = self.samples[index]
        #sample = self.loader(path)
        #print("Sample: ",sample)
        

        hm_transform = transforms.Compose([
                          transforms.Resize((224,224)),
                          transforms.ToTensor()

                                ])

        #print(path)
        #print(type(path))
        #all_images = []
        #all_labels = []
        #all_hm_face = []
        #all_hm_hand = []
        # target_acquired - 0: image path -- 1: label
        #for i in len(path):

        img_path = path
        #print("--i: %d , TA: %s"%(i,target_acquired))

        
        #img_path = target_acquired
        #label =  target_acquired[1]

        #print(img_path)

        image = Image.open(img_path)
        img_w , img_h = image.size

    
        filename = os.path.basename(img_path)
        ann_path = img_path.replace(filename, filename.split('.')[0] + '.txt')
        hm_face , hm_hand = self.get_heatmap(ann_path,img_w,img_h)  

        #image = np.dstack((image, hm_face))

        
        # Transforms
        if self.transform is not None:
            
            image = self.transform(image)
            hm_face = hm_transform(hm_face)

        image = torch.cat((image,hm_face))
        #if self.target_transform is not None:
        #    target = self.target_transform(target)
        
        
        # For Face
            #image = np.dstack((image, hm_face))

            #all_images.append(img_path)
            #all_labels.append(label)
            #all_hm_face.append(hm_face)
            #all_hm_hand.append(hm_hand)

        #return path, all_hm_face, all_hm_hand, target
        
        return image, target

    def get_heatmap(self,ann_path, img_w , img_h):
        
        hm_0 = Image.new('L', (img_w, img_h))
        hm_1 = Image.new('L', (img_w, img_h))
        
        try:

            with open(ann_path) as f:

                for line in f.readlines():

                    if line == '\n':
                        continue

                    line = line.split(' ')

                    cx = float(line[1])
                    cy = float(line[2])
                    w = float(line[3])
                    h = float(line[4])

                    xmin =  (cx - (w/2)) * img_w
                    ymin = (cy - (h/2)) * img_h
                    xmax = (cx + (w/2)) * img_w
                    ymax = (cy + (h/2)) * img_h

                    if line[0] == '0':
                        hmDraw = ImageDraw.Draw(hm_0)   
                    else:
                        hmDraw = ImageDraw.Draw(hm_1)   
                    
                    hmDraw.rectangle([xmin,ymin,xmax,ymax], fill ="#ffffff")
        except FileNotFoundError:
            print("FileNotFoundError: %s"%ann_path)
            #pass

        return hm_0, hm_1
"""
Test Code


path, target = self.samples[index]
sample = self.loader(path)
if self.transform is not None:
  sample = self.transform(sample)
if self.target_transform is not None:
  target = self.target_transform(target)
# this is where you load your resnet data
resnet_path = os.path.join(os.path.splitext(path)[0], '.pth')  # replace image extension with .pth
resnet = torch.load(resnet_path)  # load the stored features
return sample, resnet, target
"""
#dataset = MyImageFolder("falling_dataset/val/")
#re, labels = dataset[1:3]
#print(re)
#print(labels)
#re[2].save('pred_face.jpeg')
#re[3].save('pred_hand.jpeg')
input_path = "/content/drive/My Drive/Resnet_Beta-Version/dataset/alpha/Choke/"

image_datasets = {
    'train': 
    #datasets.ImageFolder(input_path + 'train', data_transforms['train']),
    MyImageFolder(input_path + 'train', data_transforms['train']),
    #ImageFolderDataset(input_path + 'train',data_transforms['train'] ),
    #FallingDataset(input_path+'train',data_transforms['train']),
    'validation': 
    MyImageFolder(input_path + 'val', data_transforms['validation']),
     'test': 
    MyImageFolder(input_path + 'test', data_transforms['test'])
    #ImageFolderDataset(input_path + 'val',data_transforms['validation'] )
}

dataloaders = {
    'train':
    torch.utils.data.DataLoader(image_datasets['train'],
                                batch_size=64,
                                shuffle=True,
                                num_workers=1),  # for Kaggle
    'validation':
    torch.utils.data.DataLoader(image_datasets['validation'],
                                batch_size=1,
                                shuffle=False,
                                num_workers=1),  # for Kaggle
     'test':
    torch.utils.data.DataLoader(image_datasets['test'],
                                batch_size=1,
                                shuffle=False,
                                num_workers=1)  # for Kaggle
}

In [0]:

epoch = 99
device = 'cuda'
model = torch.load('/content/drive/My Drive/Resnet/weights/raw-face/fallweightsresNetEpoch_%d.pth'%(epoch)).to(device)
optimizer = optim.Adam(model.fc.parameters(),lr=0.01)
model.eval()
predictions = []
target = []

iteration = 0

for inputs , labels in dataloaders['test']:

    #if iteration == 150:
    #  break
    iteration += 1
    inputs = inputs.to(device)
    labels = labels.to(device)

    
    with torch.set_grad_enabled(False):
        pred_logits_tensorFalling = model(inputs)
        predictions.append(pred_logits_tensorFalling)
        target.append(labels)

    print("iteration"+str(iteration))


In [0]:
out = []

for pred in predictions:

    out.append( torch.argmax(pred).item()) 

acc = 0
for i,tar in enumerate(target):
    if tar.item() == out[i]:
      acc += 1

print("acc", acc)
print("total",len(out))
print(acc/len(out))

acc 600
total 1108
0.5415162454873647


# Test 5 (RAW)

In [0]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ]),
    'validation':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]),
    'test':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]),
}


input_path = "/content/drive/My Drive/AlphaPose/AlphaPose-pytorch/fi_data"
image_datasets = {
    'train': 
    datasets.ImageFolder(input_path + '/train', data_transforms['train']),
    'validation': 
    datasets.ImageFolder(input_path + '/val', data_transforms['validation']),
    'test': 
    datasets.ImageFolder(input_path + '/test111', data_transforms['test'])
}

dataloaders = {
    'train':
    torch.utils.data.DataLoader(image_datasets['train'],
                                batch_size=64,
                                shuffle=True,
                                num_workers=8),  # for Kaggle
    'validation':
    torch.utils.data.DataLoader(image_datasets['validation'],
                                batch_size=1,
                                shuffle=False,
                                num_workers=8),  # for Kaggle
    'test':
    torch.utils.data.DataLoader(image_datasets['test'],
                                batch_size=1,
                                shuffle=False,
                                num_workers=8)  # for Kaggle
}

In [0]:
dataloaders['test'].dataset.classes

['Falling', 'NonFalling']

In [0]:

epoch = 33
device = 'cuda'
model = torch.load('/content/drive/My Drive/Resnet/clone_weights/raw/Epoch_%d.pth'%(epoch)).to(device)
optimizer = optim.Adam(model.fc.parameters(),lr=0.01)
model.eval()
predictions = []
target = []

iteration = 0

for inputs , labels in dataloaders['test']:

    #if iteration == 150:
    #  break
    iteration += 1
    inputs = inputs.to(device)
    labels = labels.to(device)

    
    with torch.set_grad_enabled(False):
        pred_logits_tensorFalling = model(inputs)
        predictions.append(pred_logits_tensorFalling)
        target.append(labels)

    print("iteration"+str(iteration))


iteration1
iteration2
iteration3
iteration4
iteration5
iteration6


In [0]:
out = []

for pred in predictions:

    out.append( torch.argmax(pred).item()) 

acc = 0
for i,tar in enumerate(target):
    if tar.item() == out[i]:
      acc += 1

print("acc", acc)
print("total",len(out))
print(acc/len(out))

acc 0
total 6
0.0


# TEST 6


In [0]:
%cd drive/My Drive/Resnet_Beta-Version
!ls

/content/drive/My Drive/Resnet_Beta-Version
 alpha-dataset
'alpha-Resnet(Raw+Face+Hand).ipynb'
'alpha-Resnet(Raw+Face+Hand+Pose)-Pose as Depth.ipynb'
'alpha-Resnet(Raw+Face).ipynb'
'alpha-Resnet(Raw+Hand).ipynb'
'alpha-Resnet(Raw).ipynb'
'Arrange Dataset.ipynb'
 Arslan_label_falling
'beta-Resnet(Raw+Face+Hand).ipynb'
'beta-Resnet(Raw+Face+Hand+Pose)-Pose as Depth.ipynb'
'beta-Resnet(Raw+Face).ipynb'
'beta-Resnet(Raw+Hand).ipynb'
'beta-Resnet(Raw).ipynb'
 colored_dataset_Arslan
 coloured_dataset_Govi
 extract_faceandhand_jorehgaye.ipynb
 Found_missing_detections
 GrayScale_dataset_Arslan
 GrayScale_dataset_Govi
 loss
 MissingDetections_Arslan_colored
 MissingDetections_Govi_coloured
 putTXT_rightplace.ipynb
 TestingModels_first.ipynb
 TestingModels_second.ipynb
 weights


In [0]:

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        # normalize
    ]),
    'validation':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        # normalize
    ]),
    'test':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        # normalize
    ])
}

class MyImageFolder(datasets.ImageFolder):
    
    def __init__(self, root, transform=None):
        super(MyImageFolder,self).__init__(root)

        self.transform = transform

    def __getitem__(self, index):

        

        path, target = self.samples[index]
       

        hm_transform = transforms.Compose([
                          transforms.Resize((224,224)),
                          transforms.ToTensor()

                                ])


        img_path = path
      

        image = Image.open(img_path)
        img_w , img_h = image.size

    
        filename = os.path.basename(img_path)
        ann_path = img_path.replace(filename, filename.split('.')[0] + '.txt')
        hm_face , hm_hand = self.get_heatmap(ann_path,img_w,img_h)  

        if self.transform is not None:
            
            image = self.transform(image)
            hm_face = hm_transform(hm_face)

        image = torch.cat((image,hm_face))
        
        return image, target,img_path

    def get_heatmap(self,ann_path, img_w , img_h):
        
        hm_0 = Image.new('L', (img_w, img_h))
        hm_1 = Image.new('L', (img_w, img_h))
        
        try:

            with open(ann_path) as f:

                for line in f.readlines():

                    line = line.split(' ')

                    cx = float(line[1])
                    cy = float(line[2])
                    w = float(line[3])
                    h = float(line[4])

                    xmin =  (cx - (w/2)) * img_w
                    ymin = (cy - (h/2)) * img_h
                    xmax = (cx + (w/2)) * img_w
                    ymax = (cy + (h/2)) * img_h

                    if line[0] == '0':
                        hmDraw = ImageDraw.Draw(hm_0)   
                    else:
                        hmDraw = ImageDraw.Draw(hm_1)   
                    
                    hmDraw.rectangle([xmin,ymin,xmax,ymax], fill ="#ffffff")
        except FileNotFoundError:
            #print("FileNotFoundError: %s"%ann_path)
            pass

        return hm_0, hm_1

input_path = './alpha-dataset'

image_datasets = {
    # 'train': 
    # MyImageFolder(input_path + '/train_m', data_transforms['train']),
    # 'validation': 
    # MyImageFolder(input_path + '/val_m', data_transforms['validation']),
    'test':
    MyImageFolder(input_path + '/Demo/test', data_transforms['test'])
}

dataloaders = {
    # 'train':
    # torch.utils.data.DataLoader(image_datasets['train'],
    #                             batch_size=64,
    #                             shuffle=True,
    #                             num_workers=8),  # for Kaggle
    # 'validation':
    # torch.utils.data.DataLoader(image_datasets['validation'],
    #                             batch_size=1,
    #                             shuffle=False,
    #                             num_workers=8),  # for Kaggle
    'test':
    torch.utils.data.DataLoader(image_datasets['test'],
                                batch_size=1,
                                shuffle=False,
                                num_workers=8)  # for Kaggle
}

In [0]:

model = torch.load('/content/drive/My Drive/Resnet_Beta-Version/weights/alpha/raw-face/epoch_20.pth').to(device)
optimizer = optim.Adam(model.fc.parameters(),lr=0.01)
model.eval()
paths = []
predictions = []
target = []

iteration = 0

for inputs , labels,path in dataloaders['test']:

    #if iteration == 150:
    #  break
    iteration += 1
    inputs = inputs.to(device)
    labels = labels.to(device)

    print ("path ",path)
    with torch.set_grad_enabled(False):
        pred_logits_tensorFalling = model(inputs)
        print ("results ",labels)
        predictions.append(pred_logits_tensorFalling)
        target.append(labels)
        paths.append(path)

    print("iteration"+str(iteration))


for i in range(len(paths)):
    
    print(paths[i][0])
    img = cv2.imread(paths[i][0])

    values, indices = predictions[i].max(1)
    print(indices)
    if indices == 0:
        lbl = 'Falling'
        endpoint = (320,150)
    else:
        lbl = 'Non Falling'
        endpoint = (550,150)

    img = cv2.rectangle(img, (10,10), endpoint, (255,255,255), -1)

    cv2.putText(img,lbl, 
    bottomLeftCornerOfText, 
    font, 
    fontScale,
    fontColor,
    lineType,
    thickness)

    cv2.imwrite( './frames/' + paths[i][0].split('/')[-1] , img)


path  ('./alpha-dataset/Demo/test/Falling_IMG_5730/IMG_5730_Frame132.jpg',)
results  tensor([0], device='cuda:0')
iteration1
path  ('./alpha-dataset/Demo/test/Falling_IMG_5730/IMG_5730_Frame136.jpg',)
results  tensor([0], device='cuda:0')
iteration2
path  ('./alpha-dataset/Demo/test/Falling_IMG_5730/IMG_5730_Frame140.jpg',)
results  tensor([0], device='cuda:0')
iteration3
path  ('./alpha-dataset/Demo/test/Falling_IMG_5730/IMG_5730_Frame144.jpg',)
results  tensor([0], device='cuda:0')
iteration4
path  ('./alpha-dataset/Demo/test/Falling_IMG_5730/IMG_5730_Frame148.jpg',)
results  tensor([0], device='cuda:0')
iteration5
path  ('./alpha-dataset/Demo/test/Falling_IMG_5730/IMG_5730_Frame152.jpg',)
results  tensor([0], device='cuda:0')
iteration6
path  ('./alpha-dataset/Demo/test/Falling_IMG_5730/IMG_5730_Frame156.jpg',)
results  tensor([0], device='cuda:0')
iteration7
path  ('./alpha-dataset/Demo/test/Falling_IMG_5730/IMG_5730_Frame164.jpg',)
results  tensor([0], device='cuda:0')
iteration8


In [0]:
import cv2

# Create a black image 1072x1920
img = cv2.imread('./alpha-dataset/Demo/test/Falling_IMG_5730/IMG_5730_Frame132.jpg')
#img = np.zeros((512,512,3), np.uint8)

# Write some Text

font                   = cv2.FONT_HERSHEY_SIMPLEX
bottomLeftCornerOfText = (20,100)
fontScale              = 3
fontColor              = (0,128,0)
lineType               = 2
thickness              = -1

#image = cv2.rectangle(img, (10,10), (320,150), (255,255,255), -1)   # Falling
image = cv2.rectangle(img, (10,10), (550,150), (255,255,255), -1)

cv2.putText(img,'Non Falling', 
    bottomLeftCornerOfText, 
    font, 
    fontScale,
    fontColor,
    lineType,
    thickness)



#Display the image
#cv2.imshow("img",img)

#Save image
cv2.imwrite("out.jpg", img)

True

In [0]:
out = []

for pred in predictions:

    out.append( torch.argmax(pred).item()) 

acc = 0
for i,tar in enumerate(target):
    if tar.item() == out[i]:
      acc += 1

print("acc", acc)
print("total",len(out))
print(acc/len(out))

acc 18
total 19
0.9473684210526315
